In [1]:
from gamestate import GameState, start_ai_only_game
from models.RandomModel import RandomModel
from models.HeuristicModel import HeuristicModel
from models.DeepRLModel import DeepRLModel
from trainutil import train_models, test_models
import fsutils as fs

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
env = GameState()
deep_rl_model = DeepRLModel()
rand_model_1 = RandomModel(min_steps=5, max_steps=10)
# rand_model_2 = RandomModel(min_steps=5, max_steps=10)
models = [deep_rl_model, rand_model_1]

print(deep_rl_model.model)

DQN(
  (fc1): Linear(in_features=46, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=8, bias=True)
  (relu): ReLU()
)


In [ ]:
train_models(env, models, episodes=10, steps=2000, print_every=500)
# test_models(env, models, steps=2000)

TRAIN mode
[0] [FOOD] Agent0 ate food item 264
[0] [FOOD] Agent0 ate food item 327
[0] [FOOD] Agent1 ate food item 308
[0] [FOOD] Agent1 ate food item 357
----STEP 0 rewards----
Model 0: 1.9799999999999898
Model 1: 1.9799999999999898
[4] [FOOD] Agent0 ate food item 248
[13] [FOOD] Agent1 ate food item 396
[16] [FOOD] Agent0 ate food item 326
[19] [FOOD] Agent1 ate food item 256
[31] [FOOD] Agent1 ate food item 397
[59] [FOOD] Agent0 ate food item 252
[61] [VIRUS] Agent0 ate virus 7
[64] [FOOD] Agent1 ate food item 277
[64] [FOOD] Agent1 ate food item 334
[85] [FOOD] Agent1 ate food item 317
[87] [FOOD] Agent1 ate food item 297
[92] [FOOD] Agent1 ate food item 340
[96] [FOOD] Agent0 ate food item 354
[100] [FOOD] Agent1 ate food item 331
[102] [FOOD] Agent1 ate food item 220
[113] [FOOD] Agent1 ate food item 364
[116] [FOOD] Agent1 ate food item 212
[137] [FOOD] Agent0 ate food item 361
[142] [FOOD] Agent0 ate food item 251
[168] [FOOD] Agent0 ate food item 384
[177] [FOOD] Agent0 ate f

[223] [CELL] Agent0 ate one of Agent1's cells
[230] [CELL] Agent0 ate one of Agent1's cells
[236] [CELL] Agent0 ate one of Agent1's cells
[265] [FOOD] Agent0 ate food item 833
[267] [CELL] Agent0 ate one of Agent1's cells
[268] [FOOD] Agent0 ate food item 881
[284] [FOOD] Agent1 ate food item 981
[292] [FOOD] Agent1 ate food item 978
[356] [CELL] Agent0 ate one of Agent1's cells
[357] [CELL] Agent0 ate one of Agent1's cells
[357] [GAME] Agent1 died! Was eaten by Agent0
------EPISODE 2 rewards------
Model 0: 326.49299984674684
Model 1: -249.29657076320402
[0] [FOOD] Agent0 ate food item 1003
[0] [FOOD] Agent1 ate food item 1178
----STEP 0 rewards----
Model 0: 0.9799999999999898
Model 1: 0.9799999999999898
[19] [FOOD] Agent0 ate food item 1085
[25] [FOOD] Agent1 ate food item 1011
[39] [FOOD] Agent1 ate food item 1096
[69] [FOOD] Agent1 ate food item 1176
[72] [FOOD] Agent1 ate food item 1071
[101] [FOOD] Agent1 ate food item 1022
[101] [FOOD] Agent1 ate food item 1109
[114] [FOOD] Agent

[190] [FOOD] Agent0 ate food item 1563
[200] [FOOD] Agent0 ate food item 1535
[202] [FOOD] Agent0 ate food item 1376
[202] [FOOD] Agent0 ate food item 1406
[206] [FOOD] Agent0 ate food item 1415
[206] [FOOD] Agent0 ate food item 1522
[210] [FOOD] Agent0 ate food item 1327
[210] [FOOD] Agent1 ate food item 1494
[212] [FOOD] Agent0 ate food item 1453
[212] [FOOD] Agent1 ate food item 1429
[212] [FOOD] Agent1 ate food item 1447
[213] [FOOD] Agent1 ate food item 1507
[214] [FOOD] Agent0 ate food item 1362
[214] [FOOD] Agent0 ate food item 1459
[217] [FOOD] Agent1 ate food item 1566
[220] [FOOD] Agent1 ate food item 1389
[222] [FOOD] Agent0 ate food item 1319
[224] [FOOD] Agent0 ate food item 1341
[225] [FOOD] Agent0 ate food item 1455
[228] [FOOD] Agent1 ate food item 1579
[229] [FOOD] Agent1 ate food item 1434
[230] [FOOD] Agent0 ate food item 1371
[230] [FOOD] Agent1 ate food item 1385
[231] [FOOD] Agent1 ate food item 1565
[233] [FOOD] Agent0 ate food item 1492
[245] [FOOD] Agent1 ate f

[174] [FOOD] Agent0 ate food item 2146
[197] [FOOD] Agent0 ate food item 2228
[208] [FOOD] Agent0 ate food item 2327
[228] [FOOD] Agent1 ate food item 2147
[231] [FOOD] Agent1 ate food item 2279
[232] [FOOD] Agent1 ate food item 2172
[236] [FOOD] Agent0 ate food item 2310
[237] [FOOD] Agent0 ate food item 2186
[237] [FOOD] Agent1 ate food item 2198
[243] [FOOD] Agent0 ate food item 2261
[249] [FOOD] Agent1 ate food item 2343
[252] [FOOD] Agent0 ate food item 2303
[259] [FOOD] Agent0 ate food item 2268
[269] [FOOD] Agent0 ate food item 2250
[327] [FOOD] Agent1 ate food item 2205
[367] [FOOD] Agent1 ate food item 2174
[375] [FOOD] Agent1 ate food item 2338
[383] [FOOD] Agent1 ate food item 2273
[398] [FOOD] Agent1 ate food item 2344
[399] [FOOD] Agent1 ate food item 2152
[399] [FOOD] Agent1 ate food item 2168
[404] [FOOD] Agent1 ate food item 2162
[429] [FOOD] Agent1 ate food item 2336
[432] [FOOD] Agent1 ate food item 2352
[444] [FOOD] Agent1 ate food item 2271
[444] [FOOD] Agent1 ate f

In [5]:
fs.save_net_to_disk(deep_rl_model.model, "test_drl_7")